### Introduction
In the algorithmic trading world, parameterization is so common that some algorithms are configured with hundreds or thousands of parameters. An algorithmic parent order can easily has hundreds of parameters and some of which are exposed to algorithm users so that they can update the order attributes. For example, the participation rate of a POV (Percent Of Volume) order can be amended by end users at any time during the trading duration. A common question encountered in the analysis of algorithm and/or smart order router behavior is

- what parameters are modified, and
- how they are updated  

when an amend event is initiated by an algorithmic trader.  

Below is an example history of a few selected parameters of a POV order. Create a table to show the parameter history with each parameter name as the column name in the table. You need to treat the change in order quantity and/or price limit the same as parameter change.

In [1]:
paramTbl:([]time:`time$();orderQty:`long$();limitPrice:`float$();params:());
`paramTbl insert (09:30:56.123;500000;0n;`StartTime`PovRate!(10:00:00.000;0.08));
`paramTbl insert (09:35:44.735;500000;0n;`StartTime`PovRate!(09:40:00.000;0.08));
`paramTbl insert (10:01:25.941;500000;0n;`StartTime`PovRate!(09:40:00.000;0.12));
`paramTbl insert (10:10:32.356;500000;0n;`StartTime`PovRate`MinPovRate`MaxPovRate!(09:40:00.000;0.12;0.10;0.14));
`paramTbl insert (10:30:39.475;500000;45.23;`StartTime`PovRate`MinPovRate`MaxPovRate!(09:40:00.000;0.12;0.10;0.14));
`paramTbl insert (11:00:52.092;600000;45.27;`StartTime`PovRate`MinPovRate`MaxPovRate!(09:40:00.000;0.12;0.10;0.14));
`paramTbl insert (11:00:52.092;1000000;0n;`StartTime`PovRate!(09:40:00.000;0.15));

In [23]:
paramTbl

time         orderQty limitPrice params                                      ..
-----------------------------------------------------------------------------..
09:30:56.123 500000              `StartTime`PovRate!(10:00:00.000;0.08)      ..
09:35:44.735 500000              `StartTime`PovRate!(09:40:00.000;0.08)      ..
10:01:25.941 500000              `StartTime`PovRate!(09:40:00.000;0.12)      ..
10:10:32.356 500000              `StartTime`PovRate`MinPovRate`MaxPovRate!(09..
10:30:39.475 500000   45.23      `StartTime`PovRate`MinPovRate`MaxPovRate!(09..
11:00:52.092 600000   45.27      `StartTime`PovRate`MinPovRate`MaxPovRate!(09..
11:00:52.092 1000000             `StartTime`PovRate!(09:40:00.000;0.15)      ..


### Answer
The answer below shows a few tips on dictionary manipulation:

- Use raze to merge a list of dictionaries with values of possible different types into a single dictionary
- Find the proper null values for each dictionary key
- Merge two dictionaries with join operator
- Merge two tables side by side

In [22]:
allParams:raze exec params from paramTbl;
allParams
allParams[`StartTime]
show nullValues:(key allParams)!(enlist each value allParams)[;1];

StartTime | 09:40:00.000
PovRate   | 0.15
MinPovRate| 0.1
MaxPovRate| 0.14


StartTime | 0Nt
PovRate   | 0n
MinPovRate| 0n
MaxPovRate| 0n


09:40:00.000


In [20]:
show params1:select time,OrderQty:orderQty,LimitPrice:limitPrice from paramTbl;
params2:exec {x,y}[nullValues;] each params from paramTbl;
params1,'params2

time         OrderQty LimitPrice
--------------------------------
09:30:56.123 500000             
09:35:44.735 500000             
10:01:25.941 500000             
10:10:32.356 500000             
10:30:39.475 500000   45.23     
11:00:52.092 600000   45.27     
11:00:52.092 1000000            


time         OrderQty LimitPrice StartTime    PovRate MinPovRate MaxPovRate
---------------------------------------------------------------------------
09:30:56.123 500000              10:00:00.000 0.08                         
09:35:44.735 500000              09:40:00.000 0.08                         
10:01:25.941 500000              09:40:00.000 0.12                         
10:10:32.356 500000              09:40:00.000 0.12    0.1        0.14      
10:30:39.475 500000   45.23      09:40:00.000 0.12    0.1        0.14      
11:00:52.092 600000   45.27      09:40:00.000 0.12    0.1        0.14      
11:00:52.092 1000000             09:40:00.000 0.15                         


In [8]:
{[paramTbl]
  / Merge all parameters into a single dictionary so as to get a full list of parameter names
  allParams:raze exec params from paramTbl;

  / Find the null value for each parameter name
  nullValues:(key allParams)!(enlist each value allParams)[;1];

  / Create a table of "parameter" change history of order qty and limit price
  params1:select time,OrderQty:orderQty,LimitPrice:limitPrice from paramTbl;

  / Set a proper null value to parameters that are not present
  params2:exec {x,y}[nullValues;] each params from paramTbl;

  / Combine the normal parameters with the pseudo parameters
  params1,'params2
 }[paramTbl]

time         OrderQty LimitPrice StartTime    PovRate MinPovRate MaxPovRate
---------------------------------------------------------------------------
09:30:56.123 500000              10:00:00.000 0.08                         
09:35:44.735 500000              09:40:00.000 0.08                         
10:01:25.941 500000              09:40:00.000 0.12                         
10:10:32.356 500000              09:40:00.000 0.12    0.1        0.14      
10:30:39.475 500000   45.23      09:40:00.000 0.12    0.1        0.14      
11:00:52.092 600000   45.27      09:40:00.000 0.12    0.1        0.14      
11:00:52.092 1000000             09:40:00.000 0.15                         
